<a href="https://colab.research.google.com/github/zap-frs/myrepo/blob/main/ask_law_cdqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook [1]: First steps with cdQA

This notebook shows how to use the `cdQA` pipeline to perform question answering on a custom dataset.

***Note:*** *If you are using colab, you will need to install `cdQA` by executing `!pip install cdqa` in a cell.*

In [2]:
!pip install cdqa

     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 102kB 7.7MB/s 
     |████████████████████████████████| 286kB 37.7MB/s 
     |████████████████████████████████| 10.4MB 35.8MB/s 
     |████████████████████████████████| 317kB 45.7MB/s 
     |████████████████████████████████| 6.7MB 2.2MB/s 
     |████████████████████████████████| 748.9MB 22kB/s 
     |████████████████████████████████| 92kB 11.0MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 133kB 47.4MB/s 
     |████████████████████████████████| 901kB 41.4MB/s 
     |████████████████████████████████| 1.2MB 43.6MB/s 
     |████████████████████████████████| 7.4MB 22.5MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
  Created wheel for cdqa: filename=cdqa-1.3.9-cp37-none-any.whl size=47640 sha256=b407fbcd77a95ea2e1b506b5cede4a5939c06be30e0b00d0fb32b3a698185a73
  Stored in directory: /root/.cache/pip/wheels/8b/9a/68/d3f7651ea29c30d1bebc9

In [1]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline

/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### Download pre-trained reader model and example dataset


In [2]:
from cdqa.utils.download import download_model, download_bnpp_data

download_bnpp_data(dir='http://constructiontrack1ng.com/data/')
download_model(model='bert-squad_1.1', dir='./models')

### Visualize the dataset

In [53]:
df = pd.read_csv('http://constructiontrack1ng.com/data/ffask43-Copy2.csv', converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)
df.head()

,date,title,category,link,abstract,paragraphs
0,11.01.2016,G.R. No. 167333 PEDRO LADINES Petitioner v. PE...,Press release,https://www.chanrobles.com/cralaw/2016januaryd...,NaN,[We point out that the concept of newly-discov...
1,11.01.2016,G.R. No. 203882 LORELEI O. ILADAN Petitioner v...,Press release,https://www.chanrobles.com/cralaw/2016januaryd...,NaN,"[By this Petition for Review on Certiorari,1 p..."
2,11.01.2016,G.R. No. 198450 PEOPLE OF THE PHILIPPINES Plai...,Press release,https://www.chanrobles.com/cralaw/2016januaryd...,NaN,[Their observance is the key to the successful...
3,11.01.2016,G.R. No. 197825 CAMILO SIBAL Petitioner v. PED...,Press release,https://www.chanrobles.com/cralaw/2016januaryd...,NaN,[A petition for annulment of judgment is a rem...
4,27.01.2016,G.R. No. 180993 REPUBLIC OF THE PHILIPPINES R...,Press release,https://www.chanrobles.com/cralaw/2016januaryd...,NaN,[The power of the OSG to deputize legal office...


In [52]:
from cdqa.utils.converters import df2squad

json_data = df2squad(df=df, squad_version='v1.1', output_dir='./sample_data', filename='dataset_anong')

7it [00:00, 1599.75it/s]


### Instantiate the cdQA pipeline from a pre-trained reader model

In [49]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib')
cdqa_pipeline.fit_retriever(df=df)

100%|██████████| 231508/231508 [00:00<00:00, 22933207.30B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

### Execute a query

In [58]:
from cdqa.utils.evaluation import evaluate_pipeline

evaluate_pipeline(cdqa_pipeline, '/content/sample_data/cdqa-v1.1.json')



Evaluation results: {'exact_match': 25.0, 'f1': 47.69512159218041}


{'exact_match': 25.0, 'f1': 47.69512159218041}

In [59]:
from cdqa.utils.evaluation import evaluate_reader

evaluate_reader(cdqa_pipeline, '/content/sample_data/cdqa-v1.1.json')

Evaluation expects v-1.1, but got dataset with v-v1.1


{'exact_match': 37.5, 'f1': 67.34796408985049}

In [56]:
query = ' when is certiorari available?'
prediction = cdqa_pipeline.predict(query)

### Explore predictions

In [57]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query:  when is certiorari available?
answer: when a court or other tribunal exercising quasi-judicial powers acts without or in excess of its jurisdiction
title: G.R. No. 218536 ROLANDO P. TOLENTINO Petitioner v. COMMISSION ON ELECTIONS (FIRST DIVISION)
paragraph: Certiorari is available when a court or other tribunal exercising quasi-judicial powers acts without or in excess of its jurisdiction or with grave abuse of discretion amounting to lack of jurisdiction. It is an extraordinary remedy of last resort designed to correct errors of jurisdiction.There is grave abuse of discretion justifying the issuance of the writ of certiorari when there is such capricious and whimsical exercise of judgment as is equivalent to lack of jurisdiction
